## Reversible Second Order Automata With Rule Cycling

In [1]:
import random
from bitstring import BitArray

def rule(n):
    # Exclude rules larger than 8-bits.
    assert n <= 255
    
    # Convert rule # to bits
    r = BitArray(bytes([n]))
    r.insert(~r, 0)
    
    # Return LUT
    return {x: r[x] for x in range(16)}

def pad(arr):
    # Pads the left + right bitarray sides.
    arr_padded = BitArray(arr)
    arr_padded.insert([0], 0)
    arr_padded.insert([0], len(arr_padded))
    return arr_padded

def step(arr_a, arr_b, r):
    # Apply rule
    a, b = pad(arr_a), pad(arr_b)

    row = BitArray(len(b))
    for i in range(1, len(b) - 1):
        # Check neighbors
        top, left, center, right = a[i], b[i-1], b[i], b[i+1]
        
        # Calculate next bit
        row[i] = r[15 - BitArray([top, left, center, right]).uint]

    return row

def encode(arr_a, arr_b, rules, steps):
    # Calculate rules
    rule_LUT = [rule(x) for x in rules]
    
    # Apply forward steps
    a, b = BitArray(arr_a), BitArray(arr_b)
    for i in range(steps):
        r = rule_LUT[i % len(rules)]
        c = step(a, b, r)
        a = pad(b)
        b = c
    return a, b

def decode(a, b, rules, steps):
    # Calculate rules
    rule_LUT = [rule(x) for x in rules[::-1]]
    
    # Apply reverse steps
    for i in range(steps):
        r = rule_LUT[i % len(rules)]
        c = step(b[1:-1], a[1:-1], r)
        b = a[1:-1]
        a = c[1:-1]
    return a, b

### Example 1: Encoder (single rule)

In [2]:
a = BitArray([0, 0, 0, 1, 0, 0, 0])
b = BitArray([0, 0, 1, 1, 1, 0, 0])

a_enc, b_enc = encode(a, b, [214], 2)
print(a_enc.bin)
print(b_enc.bin)

00010011000
00110011100


### Example 1: Decoder

In [3]:
a_dec, b_dec = decode(a_enc, b_enc, [214], 2)
print(a_dec.bin)
print(b_dec.bin)

0001000
0011100


### Example 2: String Encoder (multi-rule)

In [4]:
msg = 'Hello'

a = BitArray(msg.encode('utf-8'))
b = BitArray([random.getrandbits(1) for x in a])

print(a.bin)
print(b.bin)

0100100001100101011011000110110001101111
1011111010000110101111000110000110001000


In [5]:
a_enc, b_enc = encode(a, b, [45, 101], 64)
print(a_enc.hex)
print(b_enc.hex)

392554fc537e6273c884258800229cd2c89a68caf0
0eab768c33ff81a8d36db68702e7289a59d607034e


### Example 2: String Decoder

In [6]:
a_dec, b_dec = decode(a_enc, b_enc, [45, 101], 64)
a_dec.bytes.decode('utf-8')

'Hello'

### Example 3: String Encoding v2

In [7]:
a = BitArray('Hello'.encode('utf-8'))
b = BitArray([random.getrandbits(1) for x in a])
a_uint, b_uint = a.uint, b.uint

print(a_uint, b_uint)

a = BitArray(uint=a_uint, length=len(a))
b = BitArray(uint=b_uint, length=len(b))

a_enc, b_enc = encode(a, b, [45, 75, 101, 106], 64)
a_dec, b_dec = decode(a_enc, b_enc, [45, 75, 101, 106], 64)
print(a_dec.uint, a_dec.bytes.decode('utf-8'))

310939249775 843190955791
310939249775 Hello
